In [ ]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install yfiles_jupyter_graphs_for_neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
!pip install python-dotenv

In [ ]:
NEO4J_URI='neo4j+s://c16e1829.databases.neo4j.io'
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD='EsDnKL2ZIC5O4GZlu7qJqvww3ZI_igU0PHa6calBW-k'
AURA_INSTANCEID='c16e1829'
AURA_INSTANCENAME='Instance01'

In [ ]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from neo4j import GraphDatabase

from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = 'neo4j')
g = Neo4jGraphWidget(driver)

In [ ]:
default_end_date = 4102444799000

DREAM-KG

In [ ]:
create_thing_node = '''
MERGE (t:THING{
  nodeName: "THING"
}) SET t += {
  createdDate: datetime(),
  createdBy: "system",
  startTime: timestamp(),
  endTime: 4102444799000,
  nodeDesc: "RootNode",
  elementType: "class"
}
RETURN COUNT(*)
'''
create_category_node = '''
MERGE (t:THING{nodeName: "THING"})
MERGE (n:METADATA:RESOURCE{
  nodeName: "DREAM_KG",
  identifier: "dreamkg",
  abbreviation: "dreamkg",
  elementType: "class",
  nodeCategoryName: "DREAM_KG",
  metadataType: "nodeCategory"
}) SET n += {
  startTime: timestamp(),
  endTime: 4102444799000,
  nodeCategory: 1,
  createdDate: datetime(),
  createdBy: "system",
  nodeCategoryDescription: "DREAM-KG can be used to provide precision social services and resources available for people
experiencing homelessness (PEH), including emergency shelters, transitional housing, permanent
supportive housing, food assistance programs, healthcare services, employment and job training, legal
assistance, and substance abuse and mental health services."
}
MERGE (t)-[r:hasSubClass]->(n) SET r += {
  startTime: timestamp(),
  endTime: 4102444799000,
  createdBy: "system"
}
RETURN COUNT(*)
'''
def node_upsert(**kwargs):
  if not kwargs.get('subClass', False):
    label = ':DREAM_KG:RESOURCE'
    subclass = ':METADATA:RESOURCE{nodeTypeName: data.nodeType, metadataType: "nodeType"}'
    rel = 'hasInstance'
  else:
    label = ':METADATA:RESOURCE'
    subclass = ':METADATA:RESOURCE{nodeCategoryName: "DREAM_KG", metadataType: "nodeCategory"}'
    rel = 'hasSubClass'

  query = '''
  WITH $data AS data, {
    createdDate: timestamp(),
    startTime: timestamp(),
    endTime: 4102444799000,
    status: "Confirmed"
  } AS enrichData
  MATCH (nodeClass%s)'''
  if 'parent' in kwargs:
    query += ('''
    MATCH (parent:DREAM_KG:RESOURCE{identifier: "%s", abbreviation: "%s"})
    ''' % (kwargs['parent']['identifier'], kwargs['parent']['abbreviation']))
  query += '''
  MERGE (node%s{identifier: nodeClass.identifier + '.' + TOLOWER(apoc.text.regreplace(data.nodeName, '[^a-zA-Z0-9]', '')),'''
  query += ('''
                abbreviation: %s + TOLOWER(apoc.text.regreplace(data.nodeName, '[^a-zA-Z0-9]', ''))})
  ''' % ('COALESCE(parent.abbreviation, nodeClass.abbreviation)' + '+ \'.\'' if 'parent' in kwargs else '\'dreamkg.\''))
  query += '''
  ON CREATE
    SET node.operation = 'new_node'
  ON MATCH
    SET node.operation = 'update_node'
  SET node += enrichData
  SET node += data
  MERGE (node)-[:_properties{audit: true}]->(audit:AUDIT)
  CREATE (audit)-[:_time_properties]->(spoke:SPOKE) SET spoke += PROPERTIES(node)
  MERGE (node)<-[r:%s]-(nodeClass) ON CREATE SET r += enrichData
  '''
  if 'parent' in kwargs:
    query += '''
    MERGE (parent)-[pr:hasChild]->(node) ON CREATE SET pr += enrichData
    '''
  query += '''
  RETURN DISTINCT node
  '''
  query = query % (subclass, label, rel)
  return query


def business_rel_upsert(parent, rel, child):
  return '''WITH $data AS data, {
    createdDate: timestamp(),
    startTime: timestamp(),
    endTime: 4102444799000,
    status: "Confirmed"
  } AS enrichData
  MATCH (parent:DREAM_KG:RESOURCE{identifier: "%s", abbreviation: "%s"})
  MATCH (child:DREAM_KG:RESOURCE{identifier: "%s", abbreviation: "%s"})
  MERGE (parent)-[rel:%s]->(child)
  SET rel += enrichData SET rel += data
  RETURN COUNT(*)
  ''' % (parent['identifier'], parent['abbreviation'], child['identifier'], child['abbreviation'], rel)

In [ ]:
def transaction_call(tx, query):
    result = tx.run(query)
    print(list(result))
with driver.session() as session:
  session.execute_write(transaction_call, create_thing_node)
  session.execute_write(transaction_call, create_category_node)

[<Record COUNT(*)=1>]
[<Record COUNT(*)=1>]


In [ ]:
# Node Type Creation
with driver.session() as session:
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Category",
                                "isRootNodeType": True,
                                "elementType": "class",
                                "nodeTypeName": "Category",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Category",
                                "childNodeType": "Category"})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Organization",
                                "isRootNodeType": True,
                                "elementType": "class",
                                "nodeTypeName": "Organization",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "",
                                "childNodeType": "ProgramOrFacility"})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "ProgramOrFacility",
                                "elementType": "class",
                                "nodeTypeName": "ProgramOrFacility",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Organization",
                                "childNodeType": "Service"})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Service",
                                "elementType": "class",
                                "nodeTypeName": "Service",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Service",
                                "childNodeType": ""})
  session.run(node_upsert(subClass=True),
                          data={"nodeName": "Neighborhood",
                                "isRootNodeType": True,
                                "elementType": "class",
                                "nodeTypeName": "Neighborhood",
                                "nodeCategory": "DREAM_KG",
                                "metadataType": "nodeType",
                                "parentNodeType": "Neighborhood",
                                "childNodeType": "Neighborhood"})

In [ ]:
g.show_cypher('MATCH p=(:THING)-->()-->()-->()-->()-->() RETURN p')

GraphWidget(layout=Layout(height='800px', width='100%'))

Final Version - Manual Data Correction

In [ ]:
import pandas as pd
df = pd.read_csv('orgpfs-final.csv')

In [ ]:
df

,Unnamed: 0,organization,program_or_facility,service_name,_id,neighborhood,address,latitude,longitude,category,recommended_for,requirements,phone,narrative,schedule,holiday_exception
0,196,Allegheny County Department of Human Services,Allegheny Link Line,Allegheny Link line for Emergency Overnight Sh...,197,Downtown,NaN,NaN,NaN,roof-overnight,for those who are experiencing or at risk of h...,"For shelter, you will be asked to provide soci...",1-866-730-2368,"For adult singles, the Link line will provide ...",Mon - Fri: 8am - 7pm,NaN
1,142,Allegheny County Health Department,County Health Department - STD & HIV/AIDS,County STD & HIV/AIDS Clinic - STD/Hepatitis C...,143,Hill District,"1908 Wylie Avenue, PA, 15219",40.443975,-79.982066,health,all in need,"none, operates on walk-in basis",412-578-8081,Staffed by nurses and public health assistants...,"Mon, Tues, Thurs, Fri: 8:30am - 3:30pm; Wed: ...",NaN
2,145,Allegheny County Health Department,County Immunizations Clinic,County Walk-in Immunization Clinic,146,Downtown,"425 First Avenue, 4th Floor, PA, 15219",40.437083,-79.999528,health,All in need,NaN,412-578-8062,Adult Services: Provides a range of adult immu...,Mon - Tue: 9am - 4pm; Wed: 1pm - 8pm; Thu - Fr...,NaN
3,146,Allegheny County Health Department,County Pediatric Dental Program,Pediatric County Dental Clinic - McKeesport,147,McKeesport,"339 Fifth Avenue, PA, 15132",40.351186,-79.863514,health,Children 20 and younger,20 and younger,412-664-8858,Free preventative and basic restorative dental...,M-F; 8am - 4pm,NaN
4,144,Allegheny County Health Department,County Pediatric Dental Program,County Pediatric Dental Clinic - Mt. Oliver,145,Mt Oliver,"1630 Arlington Ave, PA, 15210",40.417425,-79.987510,health,all children and youth in need,"for children and youth 1 - 20 years of age, ca...",412-432-1620,Free service for children and youth. Services...,Mon - Fri: 8am - 4pm,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,4,Veterans Place,Veterans Place of Washington Blvd,"Meals, Pantry, Laundry - Veterans Place",5,Larimer,"945 Washington Blvd, PA, 15206",40.461178,-79.906674,meals|pantries-supplies,Veterans,for Vets only,412-363-0500,Great hangout space with breakfast at 8am - 9a...,Mon - Fri: 8am - 9am breakfast; Mon - Fri: ...,NaN
202,5,Veterans Place,Veterans Place of Washington Blvd,Van to Veteran's Place,6,Larimer,"945 Washington Blvd, PA, 15206",40.461178,-79.906674,activities|roof-daytime,Veterans,DD214 or Military ID,412-363-0500 X214,Call van driver (412-320-5566) ahead for morni...,NaN,NaN
203,3,Veterans Place,Veterans Place of Washington Blvd,Veterans Place,4,Larimer,"945 Washington Blvd, PA, 15206",40.461178,-79.906674,roof-daytime|finances,NaN,DD214 or Military ID required for services,412-363-0500,Veterans Place is not just a great place to ha...,Mon - Fri: 8am - 3pm,NaN
204,165,Welcome Little One,Welcome Little One,Welcome Little One,166,Downtown,"239 4th Ave STE 420, PA, 15222",40.439559,-80.001769,pantries-supplies,all in need,"for babies birth to one, no ID requirement, bu...",412-391-6862,"Diapers, baby clothes, baby food provided for ...",call for appointment,NaN


In [ ]:
def str_rem_spec_char(val: str):
  return val.replace('(', '').replace(')', '').replace('-', '').replace(' ', '') if val and pd.notna(val)  else ''

def str_strip(val):
  return val.strip() if val and pd.notna(val)  else ''

def get_node_unique_identifiers(session, node, node_type):
  query = '''
  MATCH (node:DREAM_KG:RESOURCE{nodeName: $node, nodeType: $node_type}) RETURN COLLECT({identifier: node.identifier, abbreviation: node.abbreviation}) AS dataList
  '''
  dataList = list(session.run(query, node=node, node_type=node_type))[0]['dataList']
  if len(dataList) == 1:
    return True, dataList[0]
  elif len(dataList) > 1:
    return False, dataList
  else:
    return False, None

def get_meta(node_name: str, node_type: str, cache_dict: dict):
  if node_name in cache_dict:
    res = True, cache_dict[node_name]['meta']
  else:
    res = get_node_unique_identifiers(session, node_name.strip(), node_type)
    if res[0]:
      cache_dict[node_name] = dict()
      cache_dict[node_name]['meta'] = res
    return res

In [ ]:
ORG_NODE_TYPE = 'Organization'
CAT_NODE_TYPE = 'Category'
NH_NODE_TYPE = 'Neighborhood'
PRG_NODE_TYPE = 'ProgramOrFacility'
SER_NODE_TYPE = 'Service'

root_cache_dict = dict()
root_cache_dict[ORG_NODE_TYPE] = dict()
root_cache_dict[CAT_NODE_TYPE] = dict()
root_cache_dict[NH_NODE_TYPE] = dict()
# root_cache_dict[PRG_NODE_TYPE] = dict()
# root_cache_dict[SER_NODE_TYPE] = dict()
from functools import reduce
with driver.session() as session:
  for org in df['organization'].unique():
    org_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(org),
                            "elementType": "instance",
                            "nodeType": "Organization",
                            "nodeCategory": "DREAM_KG"
                          }))[0]['node']
    root_cache_dict[ORG_NODE_TYPE][org] = {
        'identifier': org_node.get('identifier'),
        'abbreviation': org_node.get('abbreviation')
    }
  for category in set(reduce(lambda x,y: x+y, [i.split('|') for i in df['category'].unique()])):
    cat_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(category),
                            "elementType": "instance",
                            "nodeType": "Category",
                            "nodeCategory": "DREAM_KG"
                          }))[0]['node']
    root_cache_dict[CAT_NODE_TYPE][category] = {
        'identifier': cat_node.get('identifier'),
        'abbreviation': cat_node.get('abbreviation')
    }
  for neighborhood in df['neighborhood'].unique():
    nh_node = list(session.run(node_upsert(),
                          data={"nodeName": str_strip(neighborhood),
                            "elementType": "instance",
                            "nodeType": "Neighborhood",
                            "nodeCategory": "DREAM_KG"
                          }))[0]['node']
    root_cache_dict[NH_NODE_TYPE][neighborhood] = {
        'identifier': nh_node.get('identifier'),
        'abbreviation': nh_node.get('abbreviation')
    }

In [ ]:
org_dict = root_cache_dict[ORG_NODE_TYPE]
with driver.session() as session:
  for org, fac in df[['organization', 'program_or_facility']].drop_duplicates().values:
    print(org, fac)
    fac_node = list(session.run(node_upsert(parent=org_dict[str_strip(org)]),
                          data={"nodeName": str_strip(fac),
                            "elementType": "instance",
                            "nodeType": "ProgramOrFacility",
                            "nodeCategory": "DREAM_KG"
                          }))[0]['node']
    fac_meta = {
        'identifier': fac_node.get('identifier'),
        'abbreviation': fac_node.get('abbreviation')
    }
    org_dict[org][fac] = fac_meta

Allegheny County Department of Human Services Allegheny Link Line
Allegheny County Health Department County Health Department - STD & HIV/AIDS
Allegheny County Health Department County Immunizations Clinic
Allegheny County Health Department County Pediatric Dental Program
Allegheny County Library Association (ACLA) Andrew Bayne Memorial Library
Allegheny County Library Association (ACLA) Andrew Carnegie Free Library
Allegheny County Library Association (ACLA) Avalon Public Library
Allegheny County Library Association (ACLA) Bethel Park Public Library
Allegheny County Library Association (ACLA) Braddock Carnegie Library
Allegheny County Library Association (ACLA) Brentwood Library
Allegheny County Library Association (ACLA) C.C. Mellor Memorial Library - Edgewood
Allegheny County Library Association (ACLA) Carnegie Free Library of Swissvale
Allegheny County Library Association (ACLA) Carnegie Library of McKeesport
Allegheny County Library Association (ACLA) Community Library of Alleghen

In [ ]:
org_dict = root_cache_dict[ORG_NODE_TYPE]
cat_dict = root_cache_dict[CAT_NODE_TYPE]
nh_dict = root_cache_dict[NH_NODE_TYPE]

with driver.session() as session:
  for index, row in df.iterrows():
    prg_meta = org_dict[str_strip(row['organization'])][str_strip(row['program_or_facility'])]

    serv_node = list(session.run(node_upsert(parent=prg_meta),
                      data={"nodeName": str_strip(row['service_name']),
                        "elementType": "instance",
                        "nodeType": "Service",
                        "nodeCategory": "DREAM_KG",
                        "address": str_strip(row["address"]),
                        "latitude": row["latitude"],
                        "longitude": row["longitude"],
                        "recommended_for": str_strip(row["recommended_for"]),
                        "requirements": str_strip(row["requirements"]),
                        "phone": str_strip(str_rem_spec_char(row["phone"])) if row['phone'] else '',
                        "nodeDesc": str_strip(row['narrative']),
                        "schedule": str_strip(row['schedule']),
                        "holiday_exception": str_strip(row['holiday_exception'])
                      }))[0]['node']
    serv_meta = {
        'identifier': serv_node.get('identifier'),
        'abbreviation': serv_node.get('abbreviation')
    }
    for cat in row['category'].split('|'):
      cat_meta = cat_dict[cat]
      session.run(business_rel_upsert(parent=serv_meta, child=cat_meta, rel='hasCategory'), data={})

    nh_meta = nh_dict[row['neighborhood']]
    session.run(business_rel_upsert(parent=nh_meta, child=serv_meta, rel='hasService'), data={})

In [ ]:
for i, row in df[['phone']].iterrows():
  print(row)
  str_strip(str_rem_spec_char(row["phone"])) if row['phone'] else ''

phone    1-866-730-2368
Name: 0, dtype: object
phone    412-578-8081
Name: 1, dtype: object
phone    412-578-8062
Name: 2, dtype: object
phone    412-664-8858
Name: 3, dtype: object
phone    412-432-1620
Name: 4, dtype: object
phone    412-432-1620
Name: 5, dtype: object
phone    412-766-7447
Name: 6, dtype: object
phone    412-276-3456
Name: 7, dtype: object
phone    412-761-2288
Name: 8, dtype: object
phone    412-835-2207
Name: 9, dtype: object
phone    412-351-5356
Name: 10, dtype: object
phone    412-882-5694
Name: 11, dtype: object
phone    412-731-0909
Name: 12, dtype: object
phone    412-731-2300
Name: 13, dtype: object
phone    412-215-0641
Name: 14, dtype: object
phone    724-226-3491
Name: 15, dtype: object
phone    724-226-0770
Name: 16, dtype: object
phone    412-563-4552
Name: 17, dtype: object
phone    412-828-9520
Name: 18, dtype: object
phone    412-922-6877
Name: 19, dtype: object
phone    412-531-8754
Name: 20, dtype: object
phone    412-771-1222
Name: 21, dtype: obj

- Availability
  - Open
  - Closed

- Day of the Week

- Intervals
  - IntervalConstruct

- SlotHierarchy
  - SlotConstruct


In [ ]:
import pandas as pd
schedule_df = pd.read_excel('orgpfs-final21.xlsx', header=0, skiprows=[0])

In [ ]:
import json

avail_set = set()
day = set()
intervals = set()
for _, row in schedule_df[['schedule', 'schedule_json']].iterrows():
  raw = row['schedule']
  schedule = row['schedule_json']
  if pd.isna(schedule):
    print('*' * 121)
  else:
    schedule = schedule.replace('“', '"').replace(']], [[', '], [').replace('”', '"').strip(',')
    data_list = json.loads(schedule)
    for data in data_list:
      avail_set.add(data[0].upper())
      if len(data) >= 3:
        day.add(data[1])
        for i in range(2, len(data)):
          intervals.add(data[i])
print(avail_set)
print(len(day))
print(intervals)

*************************************************************************************************************************
*************************************************************************************************************************
*************************************************************************************************************************
*************************************************************************************************************************
*************************************************************************************************************************
*************************************************************************************************************************
*************************************************************************************************************************
*************************************************************************************************************************
{'C', 'BY APPOINTMENT ON